In [54]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [59]:
price_data = pd.read_csv('stockPriceData-2.csv')
# Convert 'Date' to datetime type
price_data['Date'] = pd.to_datetime(price_data['Date'])

top_500 =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
# Filter price data for dates after '2006-01-01' and drop duplicates
price_data.drop_duplicates(['Date', 'Symbol'], inplace=True)

company_master = pd.read_csv('Companymaster_6_12_2025.csv')

yearly = pd.read_csv('Finance_cons_pl_6_12_2025.csv')
Quaterly = pd.read_csv('Quarterly_cons_17_6_2025.csv')
Finance_bs =  pd.read_csv('Finance_cons_bs_6_12_2025.csv')
cashflow =  pd.read_csv('Finance_cons_cf_6_12_2025.csv')
ratios = pd.read_csv('Finance_cons_fr_6_12_2025.csv')

rebal_dates = pd.read_excel('GrowthDateUpdated.xlsx')

rebal_dates["Date"] = pd.to_datetime(rebal_dates["Date"])
rebal=rebal_dates['Date']

Rebalance_Qtr=list(rebal_dates['Quarter'])
Rebalance_Date=list(rebal_dates['Date'])
Qtr_date_dict=dict(zip(Rebalance_Date,Rebalance_Qtr))
date_Qtr_dict=dict(zip(Rebalance_Qtr,Rebalance_Date))
top_500['Quarter']=top_500['Date'].map(Qtr_date_dict)
index_constituents_data= top_500

FCF
<!-- Cash and cash equivalents as % of total assets  -->
FCF / sales
FCF / networth 
FCF / ebidta
FCF / pat 
Cfo / sales
Cfo / networth 
Cfo / ebidta 
Cfo / pat 
<!-- FCF / tangible assets -->
Fcff  / total assets 
<!-- Cash & cash equivalents / networth -->

In [60]:
ratios = ratios[['FINCODE','Year_end','FCF_Share']]
ratios = pd.merge(ratios, company_master[['FINCODE', 'SYMBOL']], left_on='FINCODE', right_on='FINCODE')
yearly = yearly[['FINCODE','Year_end','Profit_after_tax','Net_sales']]
cashflow = cashflow[['FINCODE','Year_end','Cash_from_Operation']]
data = pd.merge(yearly, cashflow, on=['FINCODE','Year_end'])
data = pd.merge(data, company_master[['FINCODE', 'SYMBOL']], left_on='FINCODE', right_on='FINCODE')
Finance_bs = Finance_bs[['Fincode','Year_end','Total_Applications','Share_Capital','Reserve']]
Finance_bs = pd.merge(Finance_bs, company_master[['FINCODE', 'SYMBOL']], left_on='Fincode', right_on='FINCODE')

In [61]:
import pandas as pd

# Step 1: Clean and prepare Finance_bs
Finance_bs = Finance_bs[['Fincode', 'Year_end', 'Total_Applications', 'Share_Capital', 'Reserve']]
Finance_bs = pd.merge(Finance_bs, company_master[['FINCODE', 'SYMBOL']], left_on='Fincode', right_on='FINCODE')

# Drop duplicate FINCODE and rename Fincode to align with other DataFrames
Finance_bs = Finance_bs.drop(columns=['FINCODE'])
Finance_bs = Finance_bs.rename(columns={'Fincode': 'FINCODE'})

# Compute Networth
Finance_bs['Networth'] = Finance_bs['Share_Capital'] + Finance_bs['Reserve']

# Step 2: Prepare and merge all datasets
# Prepare base yearly + cashflow
data = pd.merge(
    yearly[['FINCODE', 'Year_end', 'Profit_after_tax', 'Net_sales']],
    cashflow[['FINCODE', 'Year_end', 'Cash_from_Operation']],
    on=['FINCODE', 'Year_end']
)

# Add FCF
data = pd.merge(
    data,
    ratios[['FINCODE', 'Year_end', 'FCF_Share']],
    on=['FINCODE', 'Year_end']
)

# Add Finance_bs (Networth, Total Assets)
data = pd.merge(
    data,
    Finance_bs[['FINCODE', 'Year_end', 'Total_Applications', 'Networth']],
    on=['FINCODE', 'Year_end']
)

# Add SYMBOL
data = pd.merge(
    data,
    company_master[['FINCODE', 'SYMBOL']],
    on='FINCODE'
)

# Step 3: Rename columns for clarity
data = data.rename(columns={
    'Net_sales': 'Sales',
    'Profit_after_tax': 'PAT',
    'Cash_from_Operation': 'CFO',
    'FCF_Share': 'FCF',
    'Total_Applications': 'Total_Assets'
})

# Step 4: Compute Ratios
data['FCF_to_Sales'] = data['FCF'] / data['Sales']
data['FCF_to_Networth'] = data['FCF'] / data['Networth']
data['FCF_to_PAT'] = data['FCF'] / data['PAT']

data['CFO_to_Sales'] = data['CFO'] / data['Sales']
data['CFO_to_Networth'] = data['CFO'] / data['Networth']
data['CFO_to_PAT'] = data['CFO'] / data['PAT']

# Step 5: Optional column ordering
ordered_cols = ['SYMBOL', 'Year_end', 'FCF', 'CFO', 'Sales', 'Networth', 'PAT', 'Total_Assets'] + \
               [col for col in data.columns if '_to_' in col]
data = data[ordered_cols]

In [63]:
data = data[['SYMBOL','Year_end','FCF_to_Sales','FCF_to_Networth','FCF_to_PAT','CFO_to_Sales','CFO_to_Networth','CFO_to_PAT']]
data

,SYMBOL,Year_end,FCF_to_Sales,FCF_to_Networth,FCF_to_PAT,CFO_to_Sales,CFO_to_Networth,CFO_to_PAT
0,ABB,201112,-0.000430,-0.001262,-0.017335,0.016104,0.047311,0.649667
1,ABB,201112,-0.000430,-0.001262,-0.017351,0.016110,0.047311,0.650266
2,ABB,201212,-0.001342,-0.003925,-0.072456,-0.005808,-0.016989,-0.313609
3,ABB,201212,-0.001350,-0.003925,-0.074317,-0.005843,-0.016989,-0.321665
4,AEGISLOG,200703,-0.000772,-0.001592,-0.008615,0.189508,0.390723,2.113913
...,...,...,...,...,...,...,...,...
41802,NaN,202303,0.000013,0.000061,0.000361,0.026468,0.123296,0.734148
41803,NaN,202403,0.000096,0.000353,0.002627,0.039603,0.145306,1.082708
41804,NaN,202203,1.624728,11.868402,16.607043,0.148332,1.083547,1.516170
41805,NaN,202303,1.003779,8.048301,15.637107,0.081394,0.652616,1.267972


In [64]:
GrowthDate = pd.read_excel('GrowthDateUpdated.xlsx')[['Date', 'Qtr']]
GrowthDate['Qtr'] = GrowthDate['Qtr'].astype('int')
# GrowthDate Mapping  
final_df =  pd.merge(data, GrowthDate, left_on='Year_end', right_on='Qtr').drop(columns=['Year_end', 'Qtr'])
final_df = final_df.sort_values(by='Date').reset_index(drop=True)
final_df

,SYMBOL,FCF_to_Sales,FCF_to_Networth,FCF_to_PAT,CFO_to_Sales,CFO_to_Networth,CFO_to_PAT,Date
0,GLENMARK,-3.284220e-06,-5.864714e-06,-2.620543e-05,-0.038228,-0.068264,-0.305028,2006-05-31
1,SAPL,-3.827637e-05,-1.479615e-04,-2.445575e-03,-0.018702,-0.072295,-1.194921,2006-05-31
2,BHARTIARTL,-6.891446e-08,-1.093600e-07,-3.912309e-07,0.417515,0.662552,2.370254,2006-05-31
3,PNB,4.496216e-02,4.447523e-02,2.990460e-01,1.534573,1.517954,10.206537,2006-05-31
4,OFSS,3.932542e-07,4.225025e-07,2.361876e-06,0.072799,0.078213,0.437227,2006-05-31
...,...,...,...,...,...,...,...,...
38564,ROSSARI,-2.434464e-04,-4.280044e-04,-3.713502e-03,0.066069,0.116156,1.007809,2025-06-02
38565,INDIAMART,9.718362e-03,6.280664e-03,2.450060e-02,0.448889,0.290103,1.131678,2025-06-02
38566,PEL,-4.567811e-02,-1.738600e-02,-9.663078e-01,-0.787850,-0.299871,-16.666742,2025-06-02
38567,POLYCAB,2.266409e-04,5.222740e-04,2.482791e-03,0.080708,0.185985,0.884137,2025-06-02


In [65]:
price_data['Date'] = pd.to_datetime(price_data['Date'])
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df.rename(columns={'SYMBOL' : 'Symbol'},inplace=True)
merged_df = pd.merge(final_df[['Date', 'Symbol', 'FCF_to_Sales','FCF_to_Networth','FCF_to_PAT','CFO_to_Sales','CFO_to_Networth','CFO_to_PAT']], price_data, on=['Date', 'Symbol'], how='outer')
merged_df =  merged_df.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
merged_df

,Date,Symbol,FCF_to_Sales,FCF_to_Networth,FCF_to_PAT,CFO_to_Sales,CFO_to_Networth,CFO_to_PAT,Open,High,Low,Close,Volume,Mcap
11,1995-06-16,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN,13.590561,13.812036,13.590561,13.711366,1272550.0,125404.734065
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,NaN,NaN,NaN,13.474803,13.474803,13.194078,13.317597,580400.0,80205.346611
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,NaN,NaN,NaN,35.061728,35.061728,34.671605,34.765432,500.0,67415.040000
5,1995-06-16,GRASIM,NaN,NaN,NaN,NaN,NaN,NaN,129.684348,129.684348,129.505061,129.505061,250.0,47011.311897
3,1995-06-16,BPCL,NaN,NaN,NaN,NaN,NaN,NaN,12.497917,12.497917,12.497917,12.497917,100.0,44992.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN,NaN,NaN,NaN,NaN,39.150000,39.420000,38.610000,39.050000,3150428.0,114440.158443
7592959,2025-06-13,GENUSPOWER,NaN,NaN,NaN,NaN,NaN,NaN,369.950000,382.800000,368.000000,370.500000,2990813.0,112605.359198
7594499,2025-06-13,WESTLIFE,NaN,NaN,NaN,NaN,NaN,NaN,695.000000,718.950000,691.450000,716.050000,115027.0,111658.090948
7592639,2025-06-13,CCL,NaN,NaN,NaN,NaN,NaN,NaN,800.000000,841.050000,799.950000,833.700000,201301.0,111330.230424


In [66]:
merged_df[['FCF_to_Sales','FCF_to_Networth','FCF_to_PAT','CFO_to_Sales','CFO_to_Networth','CFO_to_PAT']] = merged_df.groupby('Symbol', group_keys=False)[['FCF_to_Sales','FCF_to_Networth','FCF_to_PAT','CFO_to_Sales','CFO_to_Networth','CFO_to_PAT']].apply(lambda x: x.fillna(method='ffill'))
merged_df

,Date,Symbol,FCF_to_Sales,FCF_to_Networth,FCF_to_PAT,CFO_to_Sales,CFO_to_Networth,CFO_to_PAT,Open,High,Low,Close,Volume,Mcap
11,1995-06-16,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN,13.590561,13.812036,13.590561,13.711366,1272550.0,125404.734065
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,NaN,NaN,NaN,13.474803,13.474803,13.194078,13.317597,580400.0,80205.346611
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,NaN,NaN,NaN,35.061728,35.061728,34.671605,34.765432,500.0,67415.040000
5,1995-06-16,GRASIM,NaN,NaN,NaN,NaN,NaN,NaN,129.684348,129.684348,129.505061,129.505061,250.0,47011.311897
3,1995-06-16,BPCL,NaN,NaN,NaN,NaN,NaN,NaN,12.497917,12.497917,12.497917,12.497917,100.0,44992.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN,NaN,NaN,NaN,NaN,39.150000,39.420000,38.610000,39.050000,3150428.0,114440.158443
7592959,2025-06-13,GENUSPOWER,NaN,NaN,NaN,NaN,NaN,NaN,369.950000,382.800000,368.000000,370.500000,2990813.0,112605.359198
7594499,2025-06-13,WESTLIFE,0.000114,0.000468,0.003925,0.141965,0.584518,4.906070,695.000000,718.950000,691.450000,716.050000,115027.0,111658.090948
7592639,2025-06-13,CCL,-0.000062,-0.000087,-0.000455,0.083661,0.116565,0.610222,800.000000,841.050000,799.950000,833.700000,201301.0,111330.230424


In [67]:
merged_df = merged_df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Mcap'])
merged_df

,Date,Symbol,FCF_to_Sales,FCF_to_Networth,FCF_to_PAT,CFO_to_Sales,CFO_to_Networth,CFO_to_PAT
11,1995-06-16,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,NaN,NaN,NaN
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,NaN,NaN,NaN
5,1995-06-16,GRASIM,NaN,NaN,NaN,NaN,NaN,NaN
3,1995-06-16,BPCL,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN,NaN,NaN,NaN,NaN
7592959,2025-06-13,GENUSPOWER,NaN,NaN,NaN,NaN,NaN,NaN
7594499,2025-06-13,WESTLIFE,0.000114,0.000468,0.003925,0.141965,0.584518,4.906070
7592639,2025-06-13,CCL,-0.000062,-0.000087,-0.000455,0.083661,0.116565,0.610222


In [68]:
rank_cols = [col for col in merged_df.columns if col not in ['Date', 'Symbol']]

# Apply cross-sectional percentile rank for each date
merged_df[rank_cols] = merged_df.groupby('Date')[rank_cols].transform(lambda x: x.rank(pct=True))

merged_df

,Date,Symbol,FCF_to_Sales,FCF_to_Networth,FCF_to_PAT,CFO_to_Sales,CFO_to_Networth,CFO_to_PAT
11,1995-06-16,RELIANCE,NaN,NaN,NaN,NaN,NaN,NaN
13,1995-06-16,TATASTEEL,NaN,NaN,NaN,NaN,NaN,NaN
8,1995-06-16,HINDPETRO,NaN,NaN,NaN,NaN,NaN,NaN
5,1995-06-16,GRASIM,NaN,NaN,NaN,NaN,NaN,NaN
3,1995-06-16,BPCL,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN,NaN,NaN,NaN,NaN
7592959,2025-06-13,GENUSPOWER,NaN,NaN,NaN,NaN,NaN,NaN
7594499,2025-06-13,WESTLIFE,0.567633,0.670644,0.694511,0.553140,0.961814,0.952267
7592639,2025-06-13,CCL,0.229469,0.219570,0.221957,0.355072,0.324582,0.252983


In [77]:

# Assuming your dataframe is named df
merged_df['Date'] = pd.to_datetime(merged_df['Date'])  # Ensure datetime type

# Extract year and month
merged_df['Year'] = merged_df['Date'].dt.year
merged_df['Month'] = merged_df['Date'].dt.month

# Filter for only the desired months
key_months = [8]  # March, June, September, December
df_filtered = merged_df[merged_df['Month'].isin(key_months)].copy()

# Get the first available date per Year-Month pair
first_dates = (
    df_filtered.groupby(['Year', 'Month'])['Date']
    .min()
    .reset_index()
)

# Merge back to get the full rows corresponding to the first dates
df_first_in_months = pd.merge(
    df_filtered,
    first_dates,
    on=['Year', 'Month', 'Date'],
    how='inner'
)

# Drop helper columns if not needed
df_first_in_months = df_first_in_months.drop(columns=['Year', 'Month'])



In [78]:
df_first_in_months = df_first_in_months[df_first_in_months['Date'].dt.year >= 2006].copy()
df_first_in_months

,Date,Symbol,FCF_to_Sales,FCF_to_Networth,FCF_to_PAT,CFO_to_Sales,CFO_to_Networth,CFO_to_PAT
2307,2006-08-01,ONGC,0.823970,0.794007,0.786517,0.924812,0.857143,0.736842
2308,2006-08-01,RELIANCE,0.359551,0.363296,0.355805,0.642857,0.639098,0.627820
2309,2006-08-01,NTPC,0.385768,0.385768,0.385768,0.868421,0.500000,0.609023
2310,2006-08-01,INFY,0.970037,0.955056,0.955056,0.898496,0.812030,0.582707
2311,2006-08-01,TCS,0.947566,0.943820,0.917603,0.796992,0.887218,0.507519
...,...,...,...,...,...,...,...,...
11802,2024-08-01,RTNINDIA,0.479730,0.619154,0.501114,0.153153,0.133630,0.180401
11803,2024-08-01,ALKYLAMINE,0.722973,0.766147,0.663697,0.738739,0.866370,0.412027
11804,2024-08-01,METROPOLIS,0.637387,0.601336,0.625835,0.795045,0.674833,0.859688
11805,2024-08-01,LTFOODS,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df_first_in_months = df_first_in_months.dropna()
df_first_in_months

,Date,Symbol,FCF_to_Sales,FCF_to_Networth,FCF_to_PAT,CFO_to_Sales,CFO_to_Networth,CFO_to_PAT
2307,2006-08-01,ONGC,0.823970,0.794007,0.786517,0.924812,0.857143,0.736842
2308,2006-08-01,RELIANCE,0.359551,0.363296,0.355805,0.642857,0.639098,0.627820
2309,2006-08-01,NTPC,0.385768,0.385768,0.385768,0.868421,0.500000,0.609023
2310,2006-08-01,INFY,0.970037,0.955056,0.955056,0.898496,0.812030,0.582707
2311,2006-08-01,TCS,0.947566,0.943820,0.917603,0.796992,0.887218,0.507519
...,...,...,...,...,...,...,...,...
11801,2024-08-01,SAPPHIRE,0.653153,0.677060,0.661470,0.680180,0.812918,0.768374
11802,2024-08-01,RTNINDIA,0.479730,0.619154,0.501114,0.153153,0.133630,0.180401
11803,2024-08-01,ALKYLAMINE,0.722973,0.766147,0.663697,0.738739,0.866370,0.412027
11804,2024-08-01,METROPOLIS,0.637387,0.601336,0.625835,0.795045,0.674833,0.859688


In [91]:
top_25 = (
    df_first_in_months[['Date', 'Symbol', 'CFO_to_PAT']]
      .sort_values(['Date', 'CFO_to_PAT'], ascending=[True, False])
      .groupby('Date')
      .head(50)
      .reset_index(drop=True)
)
top_25

,Date,Symbol,CFO_to_PAT
0,2006-08-01,TATACOMM,1.000000
1,2006-08-01,PNB,0.996241
2,2006-08-01,ABIRLANUVO,0.992481
3,2006-08-01,JAICORPLTD,0.988722
4,2006-08-01,VISASTEEL,0.984962
...,...,...,...
945,2024-08-01,BAJAJELEC,0.897550
946,2024-08-01,INDIGO,0.895323
947,2024-08-01,LEMONTREE,0.893096
948,2024-08-01,JUBLFOOD,0.890869


In [92]:
top_25.to_csv('CFO_to_PAT.csv', index=False)

In [94]:
price_data = pd.read_csv('stockPriceData-2.csv')
# etf_indices = pd.read_csv('.csv')
# price_data = price_data[~(price_data['Symbol'].isin(etf_indices['Symbol']))]
price_data_500 = price_data.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
df = price_data[price_data['Symbol'].isin(price_data_500['Symbol'])]
df['Date'] = pd.to_datetime(df['Date'])
master_date = df.drop_duplicates(subset='Date')[['Date']].reset_index(drop=True)
df.set_index('Date', inplace=True)

# Step 1: Sort by Date and descending Mcap (higher Mcap = rank 1)
df = df.sort_values(by=['Date', 'Mcap'], ascending=[True, False])

# Step 2: Create daily rank
df['Daily_Mcap_Rank'] = df.groupby('Date')['Mcap'].rank(method='first', ascending=False)

# Step 3: Define classification function
def classify_mcap(rank):
    if rank <= 100:
        return 'Large'
    elif rank <= 250:
        return 'Mid'
    else:
        return 'Small'

# Step 4: Apply classification
df['Mcap_Category'] = df['Daily_Mcap_Rank'].apply(classify_mcap)
df = df.sort_values(['Symbol', 'Date'])
df['PrevClose'] = df.groupby('Symbol')['Close'].shift(1)
df['returns'] = (df['Close'] - df['PrevClose']) / df['PrevClose']

dividend =  pd.read_csv('Appender_latest-3.csv')
dividend = dividend[['Date', 'Symbol','Dividend','DownVol','Growth']]

dividend['Date'] = pd.to_datetime(dividend['Date'])
dividend = pd.merge(dividend, df[['Symbol','Mcap','Mcap_Category']], on=['Date','Symbol'])
dividend = dividend.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))

dividend

,Date,Symbol,Dividend,DownVol,Growth,Mcap,Mcap_Category
298,2006-01-03,ONGC,NaN,0.917275,NaN,1.726521e+06,Large
337,2006-01-03,RELIANCE,NaN,0.980535,NaN,1.282027e+06,Large
293,2006-01-03,NTPC,NaN,0.944039,NaN,9.243166e+05,Large
412,2006-01-03,TCS,NaN,0.802920,NaN,8.321350e+05,Large
205,2006-01-03,INFY,NaN,0.929440,NaN,8.254697e+05,Large
...,...,...,...,...,...,...,...
2288325,2025-06-13,CCL,0.557692,0.676533,0.634981,1.113302e+05,Small
2288522,2025-06-13,LEMONTREE,0.080128,0.431290,0.293539,1.097816e+05,Small
2288400,2025-06-13,GALLANTT,0.356838,0.365751,0.660354,1.086006e+05,Small
2288647,2025-06-13,SHRIPISTON,0.472222,0.463002,0.274868,1.073054e+05,Small


In [96]:
fcf = merged_df[['Date','Symbol','FCF_to_Networth','CFO_to_Networth']]
fcf

,Date,Symbol,FCF_to_Networth,CFO_to_Networth
11,1995-06-16,RELIANCE,NaN,NaN
13,1995-06-16,TATASTEEL,NaN,NaN
8,1995-06-16,HINDPETRO,NaN,NaN
5,1995-06-16,GRASIM,NaN,NaN
3,1995-06-16,BPCL,NaN,NaN
...,...,...,...,...
7593674,2025-06-13,NSLNISP,NaN,NaN
7592959,2025-06-13,GENUSPOWER,NaN,NaN
7594499,2025-06-13,WESTLIFE,0.670644,0.961814
7592639,2025-06-13,CCL,0.219570,0.324582


In [ ]:
dividend = pd.merge(dividend,fcf, on=['Date','Symbol'])
dividend

,Date,Symbol,Dividend,DownVol,Growth,Mcap,Mcap_Category,FCF_to_Networth,CFO_to_Networth
0,2006-01-03,ONGC,NaN,0.917275,NaN,1.726521e+06,Large,NaN,NaN
1,2006-01-03,RELIANCE,NaN,0.980535,NaN,1.282027e+06,Large,NaN,NaN
2,2006-01-03,NTPC,NaN,0.944039,NaN,9.243166e+05,Large,NaN,NaN
3,2006-01-03,TCS,NaN,0.802920,NaN,8.321350e+05,Large,NaN,NaN
4,2006-01-03,INFY,NaN,0.929440,NaN,8.254697e+05,Large,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2279017,2025-06-13,NSLNISP,NaN,0.482030,NaN,1.144402e+05,Small,NaN,NaN
2279018,2025-06-13,GENUSPOWER,0.080128,0.084567,0.811737,1.126054e+05,Small,NaN,NaN
2279019,2025-06-13,WESTLIFE,0.080128,0.594080,0.290075,1.116581e+05,Small,0.670644,0.961814
2279020,2025-06-13,CCL,0.557692,0.676533,0.634981,1.113302e+05,Small,0.219570,0.324582


In [98]:
# Compute means only when both values are not NaN
dividend['Div_And_CFO_to_Networth'] = dividend.apply(
    lambda row: np.mean([row['Dividend'], row['CFO_to_Networth']]) 
    if pd.notna(row['Dividend']) and pd.notna(row['CFO_to_Networth']) 
    else np.nan,
    axis=1
)

dividend['Div_And_FCF_to_Networth'] = dividend.apply(
    lambda row: np.mean([row['Dividend'], row['FCF_to_Networth']]) 
    if pd.notna(row['Dividend']) and pd.notna(row['FCF_to_Networth']) 
    else np.nan,
    axis=1
)
dividend

,Date,Symbol,Dividend,DownVol,Growth,Mcap,Mcap_Category,FCF_to_Networth,CFO_to_Networth,Div_And_CFO_to_Networth,Div_And_FCF_to_Networth
0,2006-01-03,ONGC,NaN,0.917275,NaN,1.726521e+06,Large,NaN,NaN,NaN,NaN
1,2006-01-03,RELIANCE,NaN,0.980535,NaN,1.282027e+06,Large,NaN,NaN,NaN,NaN
2,2006-01-03,NTPC,NaN,0.944039,NaN,9.243166e+05,Large,NaN,NaN,NaN,NaN
3,2006-01-03,TCS,NaN,0.802920,NaN,8.321350e+05,Large,NaN,NaN,NaN,NaN
4,2006-01-03,INFY,NaN,0.929440,NaN,8.254697e+05,Large,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2279017,2025-06-13,NSLNISP,NaN,0.482030,NaN,1.144402e+05,Small,NaN,NaN,NaN,NaN
2279018,2025-06-13,GENUSPOWER,0.080128,0.084567,0.811737,1.126054e+05,Small,NaN,NaN,NaN,NaN
2279019,2025-06-13,WESTLIFE,0.080128,0.594080,0.290075,1.116581e+05,Small,0.670644,0.961814,0.520971,0.375386
2279020,2025-06-13,CCL,0.557692,0.676533,0.634981,1.113302e+05,Small,0.219570,0.324582,0.441137,0.388631


In [107]:
top_50a = (
    dividend[['Date', 'Symbol', 'Div_And_FCF_to_Networth','DownVol','Growth']]
      .sort_values(['Date', 'Div_And_FCF_to_Networth'], ascending=[True, False])
      .groupby('Date')
      .head(100)
      .reset_index(drop=True)
)
top_50a['Date'] = pd.to_datetime(top_50a['Date'])
top_50a

,Date,Symbol,Div_And_FCF_to_Networth,DownVol,Growth
0,2006-01-03,ONGC,NaN,0.917275,NaN
1,2006-01-03,RELIANCE,NaN,0.980535,NaN
2,2006-01-03,NTPC,NaN,0.944039,NaN
3,2006-01-03,TCS,NaN,0.802920,NaN
4,2006-01-03,INFY,NaN,0.929440,NaN
...,...,...,...,...,...
479695,2025-06-13,PCBL,0.681196,0.194503,0.200720
479696,2025-06-13,UNITDSPR,0.679288,0.875264,0.487051
479697,2025-06-13,EMAMILTD,0.678447,0.399577,0.196275
479698,2025-06-13,TATACONSUM,0.677690,0.913319,0.833102


In [108]:
top_50a = top_50a.dropna()
# Function to get top 25 by Growth and DownVol for each date
def get_top_25(df):
    top_growth = df.sort_values(['Date', 'DownVol'], ascending=[True, False])\
                   .groupby('Date').head(25)\
                   .assign(RankType='TopDownVol')

    top_downvol = df.sort_values(['Date', 'Growth'], ascending=[True, False])\
                    .groupby('Date').head(25)\
                    .assign(RankType='TopGrowth')

    return pd.concat([top_growth, top_downvol]).reset_index(drop=True)

# Apply function
top_25_combined = get_top_25(top_50a)
top_25_combined

,Date,Symbol,Div_And_FCF_to_Networth,DownVol,Growth,RankType
0,2006-06-06,GODREJCP,0.701940,0.992665,0.530318,TopDownVol
1,2006-06-06,ASIANPAINT,0.694560,0.990220,0.703839,TopDownVol
2,2006-06-06,KANSAINER,0.617069,0.980440,0.564999,TopDownVol
3,2006-06-06,HDFCBANK,0.611534,0.975550,0.552110,TopDownVol
4,2006-06-06,INFY,0.722235,0.955990,0.384467,TopDownVol
...,...,...,...,...,...,...
234745,2025-06-13,GLAXO,0.730134,0.581395,0.670802,TopGrowth
234746,2025-06-13,REDINGTON,0.841888,0.317125,0.664770,TopGrowth
234747,2025-06-13,NMDC,0.765796,0.384778,0.651912,TopGrowth
234748,2025-06-13,LALPATHLAB,0.769269,0.832981,0.650514,TopGrowth


In [109]:

# Assuming your dataframe is named df
top_25_combined['Date'] = pd.to_datetime(top_25_combined['Date'])  # Ensure datetime type

# Extract year and month
top_25_combined['Year'] = top_25_combined['Date'].dt.year
top_25_combined['Month'] = top_25_combined['Date'].dt.month

# Filter for only the desired months
key_months = [8]  # March, June, September, December
df_filtered = top_25_combined[top_25_combined['Month'].isin(key_months)].copy()

# Get the first available date per Year-Month pair
first_dates = (
    df_filtered.groupby(['Year', 'Month'])['Date']
    .min()
    .reset_index()
)

# Merge back to get the full rows corresponding to the first dates
df_first_in_months = pd.merge(
    df_filtered,
    first_dates,
    on=['Year', 'Month', 'Date'],
    how='inner'
)

# Drop helper columns if not needed
df_first_in_months = df_first_in_months.drop(columns=['Year', 'Month'])
df_first_in_months


,Date,Symbol,Div_And_FCF_to_Networth,DownVol,Growth,RankType
0,2006-08-01,KANSAINER,0.616905,0.995181,0.564999,TopDownVol
1,2006-08-01,INFY,0.721774,0.990361,0.384467,TopDownVol
2,2006-08-01,HDFCBANK,0.611287,0.983133,0.552110,TopDownVol
3,2006-08-01,ASIANPAINT,0.695556,0.978313,0.703839,TopDownVol
4,2006-08-01,TCS,0.716156,0.973494,0.748238,TopDownVol
...,...,...,...,...,...,...
945,2024-08-01,FINPIPE,0.717078,0.404661,0.719614,TopGrowth
946,2024-08-01,APLLTD,0.808432,0.595339,0.718705,TopGrowth
947,2024-08-01,UTIAMC,0.968178,0.771186,0.718696,TopGrowth
948,2024-08-01,KANSAINER,0.838838,0.853814,0.712183,TopGrowth


In [110]:
# Assuming your DataFrame is called `df` and has columns ['Date', 'Symbol']
df_first_in_months['Date'] = pd.to_datetime(df_first_in_months['Date'])

# Step 1: Count occurrences per (Date, Symbol)
weight_df = df_first_in_months.groupby(['Date', 'Symbol']).size().reset_index(name='Count')

# Step 2: Compute total count per date
weight_df['Total_Count'] = weight_df.groupby('Date')['Count'].transform('sum')

# Step 3: Compute proportional weight
weight_df['Weight'] = weight_df['Count'] / weight_df['Total_Count']

# Step 4: Final columns
final_weights = weight_df[['Date', 'Symbol', 'Weight']]
final_weights

,Date,Symbol,Weight
0,2006-08-01,63MOONS,0.02
1,2006-08-01,APARINDS,0.02
2,2006-08-01,ASHAPURMIN,0.02
3,2006-08-01,ASIANPAINT,0.04
4,2006-08-01,ATUL,0.02
...,...,...,...
846,2024-08-01,TATAELXSI,0.02
847,2024-08-01,TCS,0.02
848,2024-08-01,TORNTPHARM,0.02
849,2024-08-01,TTKPRESTIG,0.02


In [111]:
final_weights.to_csv('Div_And_FCF_to_Networth.csv', index=False)